In [2]:
import openai
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys, ActionChains

driver = webdriver.Firefox(executable_path=r'/Users/rogerjunior/geckodriver')

openai.api_key = os.getenv("OPENAI_API_KEY")

/var/folders/41/t4024s9j7lsf7jdsjsqxx5280000gn/T/ipykernel_12836/2827345077.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'/Users/rogerjunior/geckodriver')


In [3]:
js_code = """function removeAttributesExceptSrcHrefAlt(element) {
  const attributes = element.attributes;
  for (let i = attributes.length - 1; i >= 0; i--) {
    const attrName = attributes[i].name;
    if (attrName !== 'src' && attrName !== 'href' && attrName !== 'alt') {
      element.removeAttribute(attrName);
    }
  }
  element.removeAttribute("class")
  element.removeAttribute("srcset")
  element.removeAttribute("style")
  element.removeAttribute("sizes")
}

function minify_html(input) {
  return input
  .replace(/\<\!--\s*?[^\s?\[][\s\S]*?--\>/g,'')
  .replace(/\>\s*\</g,'><')
  .replace(/\s\s+/g, '')
}

// Function to traverse and process all elements in the document
function processAllElements(query) {
  const allElements = document.querySelectorAll('*');

  const tagsToRemove = ["script", "svg", "style", "iframe"];

  tagsToRemove.forEach(tag => {
    document.querySelectorAll(tag).forEach(el => {
      el.remove()
    })
  });

  allElements.forEach(function (element) {
    removeAttributesExceptSrcHrefAlt(element);
  });

  return minify_html(document.querySelector(query).outerHTML);
}

processAllElements("body") """

In [4]:
driver.get("https://poshclublisbon.com/eventos/index.php")
driver.execute_script(js_code)
body = driver.find_element(by=By.TAG_NAME, value="body")
html = body.get_attribute("innerHTML")

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You will be provided with an HTML source code. Your job is to extract all events listed and then format them like a JSON array of objects. Your response should contain only the resulting JSON with the following keys and formats:\n- 'price' - look for '€', 'R$' or '$'. Format as a number like this 9.99. This should be a JSON number, which means no letters or apostrophes.\n- 'currency' - should be 'EUR', 'BRL' or 'USD'\n- 'name',\n- 'date' - in the pattern: YYYY-MM-DDTHH:MM:SS.511Z. Note that today is 2023-10-11 so format dates written as weekdays accordingly.\n- 'image',\n- 'event_url',\n- 'location'"},
        {"role": "user", "content": html},
    ],
    max_tokens=1000
)

In [7]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import firestore
from datetime import datetime
import urllib.request 
import uuid
from firebase_admin import storage

In [8]:
cred = credentials.Certificate("firebase.json")
firebase_admin.initialize_app(cred, { 'storageBucket': 'the-queer-spot.appspot.com'})

In [11]:
db = firestore.client()
events = json.loads(response.choices[0].message.content)
bucket = storage.bucket()

for event in events:
    event["createdAt"] = firestore.SERVER_TIMESTAMP

    date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 
    event["date"] = datetime.strptime(event["date"], date_format)

    url = event["image"]
    local_image = f"events/{str(uuid.uuid4())}.{url.split('.')[-1]}"
    urllib.request.urlretrieve(url, local_image) 
    blob = bucket.blob(local_image)
    blob.upload_from_filename(local_image)
    blob.make_public()
    event["image"] = blob.public_url

    doc_ref = db.collection("events").document()
    doc_ref.set(event)